In [1]:
import os
from zhipuai import ZhipuAI
from dotenv import load_dotenv, find_dotenv


# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

client = ZhipuAI(
    api_key=os.environ["ZHIPUAI_API_KEY"]
)

def gen_glm_params(prompt):
    '''
    构造 GLM 模型请求参数 messages

    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="glm-4", temperature=0.95):
    '''
    获取 GLM 模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 glm-4，也可以按需选择 glm-3-turbo 等其他模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~1.0，且不能设置为 0。温度系数越低，输出内容越一致。
    '''

    messages = gen_glm_params(prompt)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"


In [5]:
# 不使用分隔符
prompt = f"""
请生成包括书名、作者和类别的三本虚构的、非真实存在的中文书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""

# 调用 
response = get_completion(prompt)
print(response)

以下是三本虚构的、非真实存在的中文书籍清单，以 JSON 格式提供：

```json
[
  {
    "book_id": "1",
    "title": "《岁月无声》",
    "author": "李慕白",
    "genre": "历史小说"
  },
  {
    "book_id": "2",
    "title": "《星际迷航：无尽宇宙》",
    "author": "赵天行",
    "genre": "科幻小说"
  },
  {
    "book_id": "3",
    "title": "《推理至上：消失的凶手》",
    "author": "林梦溪",
    "genre": "侦探推理"
  }
]
```

请注意，上述书籍名称、作者和类别均为虚构，非真实存在。


In [6]:
# 满足条件的输入（text_1 中提供了步骤）

text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""

prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：
第一步 - ...
第二步 - …
…
第N步 - …
如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
{text_1}
"""

response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开
第二步 - 拿一个杯子并把茶包放进去
第三步 - 把热水倒在茶包上
第四步 - 等待茶叶浸泡
第五步 - 几分钟后取出茶包
第六步 - 如需调味，可加入糖或牛奶
第七步 - 享受美味的茶


In [7]:
# 不满足条件的输入（text_2 中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""

prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：
第一步 - ...
第二步 - …
…
第N步 - …
如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
{text_2}
"""

response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


In [8]:
prompt = f"""
你的任务是以一致的风格回答问题（注意：文言文和白话的区别）。
<学生>: 请教我何为耐心。
<圣贤>: 天生我材必有用，千金散尽还复来。
<学生>: 请教我何为坚持。
<圣贤>: 故不积跬步，无以至千里；不积小流，无以成江海。骑骥一跃，不能十步；驽马十驾，功在不舍。
<学生>: 请教我何为孝顺。
"""
response = get_completion(prompt)
print(response)

<圣贤>：孝顺者，尊父母之道也。夫孝，德之本也，教之所由生也。父母者，身之根本，不可不敬。养可能也，敬为难；敬可能也，安为难。是以君子行孝，无违仁之本，无失礼之节，使人以安，为之以敬，此所谓孝顺也。 

（白话翻译：孝顺是指尊敬父母的道理。孝顺是美德的根本，也是教育的源头。父母是我们的根基，我们必须尊敬他们。养育他们可能是容易的，但尊敬他们却很难；尊敬他们可能是容易的，但让他们安心却很难。所以君子实行孝道，不违背仁义的根本，不失去礼节的节制，让父母感到安心，以尊敬的态度对待他们，这就是所说的孝顺。）


In [10]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""

prompt = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。
请使用以下格式：
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON 格式：<带有 English_summary 和 num_names 的 JSON 格式>
Text: <{text}>
"""

response = get_completion(prompt)
print("response :")
print(response)

response :
摘要：在迷人村庄，杰克和吉尔兄妹俩爬山取水，不幸摔伤，但未减弱冒险精神。
翻译：In a charming village, siblings Jack and Gill set out to fetch water from a mountain well, sustaining injuries but remaining undeterred in their spirit of adventure.
名称：Jack, Gill
输出 JSON 格式：

```json
{
  "English_summary": "In a charming village, siblings Jack and Gill fetch water from a mountain well, sustaining injuries but maintaining their spirit of adventure.",
  "num_names": 2
}
```


In [11]:
prompt = f"""
判断学生的解决方案是否正确。
问题:
我正在建造一个太阳能发电站，需要帮助计算财务。
土地费用为 100美元/平方英尺
我可以以 250美元/平方英尺的价格购买太阳能电池板
我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
土地费用：100x
太阳能电池板费用：250x
维护费用：100,000美元+100x
总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""

response = get_completion(prompt)
print(response)

学生的解决方案基本上是正确的。首年运营的总费用确实应该包括土地费用、太阳能电池板费用以及维护费用。以下是每个部分的详细计算：

1. 土地费用：费用与发电站的大小直接相关，即每平方英尺100美元，所以总土地费用为 \( 100 \times x \)。

2. 太阳能电池板费用：同样，费用与发电站的大小相关，每平方英尺250美元，因此总费用为 \( 250 \times x \)。

3. 维护费用：由固定的每年10万美元加上每平方英尺10美元的费用组成，因此总维护费用为 \( 100,000 + 10 \times x \)。

综合上述三个部分的费用，首年运营的总费用确实是：

\[ 总费用 = 土地费用 + 太阳能电池板费用 + 维护费用 \]
\[ 总费用 = 100x + 250x + (100,000 + 10x) \]
\[ 总费用 = 450x + 100,000 \text{美元} \]

所以学生的计算结果 \( 450x + 100,000 \text{美元} \) 是正确的。


In [12]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：
步骤：
首先，自己解决问题。
然后将您的解决方案与学生的解决方案进行比较，对比计算得到的总费用与学生计算的总费用是否一致，
并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。
使用以下格式：
问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
学生计算的总费用：学生计算得到的总费用
实际计算的总费用：实际计算出的总费用
学生计算的费用和实际计算的费用是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否
学生的成绩：正确或不正确
问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;
作为平方英尺数的函数，首年运营的总费用是多少。
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x美元
2. 太阳能电池板费用：250x美元
3. 维护费用：100,000+100x=10万美元+10x美元
总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元
实际解决方案和步骤：
"""

response = get_completion(prompt)
print(response)

首先，需要明确问题的要求，即计算首年运营的总费用，这里的费用包括土地费用、太阳能电池板费用以及维护费用。

实际解决方案和步骤：
1. 土地费用：土地费用是每平方英尺100美元，所以土地总费用为100x美元，其中x是发电站的大小（平方英尺）。
2. 太阳能电池板费用：太阳能电池板的价格是每平方英尺250美元，因此太阳能电池板的总费用为250x美元。
3. 维护费用：维护合同包括固定费用和按平方英尺计算的额外费用。固定费用是10万美元，每平方英尺的额外费用是10美元，所以总维护费用为100,000 + 10x美元。

总费用计算如下：
总费用 = 土地费用 + 太阳能电池板费用 + 维护费用
总费用 = 100x + 250x + (100,000 + 10x)美元
总费用 = 360x + 100,000美元

学生计算的总费用：450x + 10万美元
实际计算的总费用：360x + 100,000美元

学生计算的费用和实际计算的费用是否相同：否
学生的解决方案和实际解决方案是否相同：否

学生的成绩：不正确

在学生的解决方案中，他在计算总费用时重复计算了土地费用，并且错误地将维护费用的每平方英尺部分与土地费用相加，导致计算的总费用高于实际费用。正确的计算应该把土地费用计算一次，维护费用正确地累加固定和变动部分。


In [13]:
prompt = f"""
给我一些研究LLM长度外推的论文，包括论文标题、主要内容和链接
"""

response = get_completion(prompt)
print(response)

以下是几篇研究LLM（大型语言模型）长度外推的论文，包括论文标题、主要内容和链接：

1. 论文标题：Efficient Length Extrapolation for Language Models
主要内容：本文提出了一种有效的长度外推方法，使大型语言模型能够在有限的计算资源下处理更长的序列。作者通过改进模型架构和训练策略，实现了在长文本上的出色性能。此外，论文还讨论了长度外推在语言生成、文本分类和机器翻译等任务中的应用。
链接：https://arxiv.org/abs/2009.09796

2. 论文标题：Length Extrapolation in Transformers with Continuous Representations
主要内容：本文研究了在Transformer模型中采用连续表示进行长度外推的方法。作者提出了一种新颖的注意力机制，使得模型能够处理比训练时更长的序列。此外，论文还通过实验验证了该方法在多个自然语言处理任务中的有效性。
链接：https://arxiv.org/abs/2102.05095

3. 论文标题：Generalized Length Extrapolation for Pre-trained Language Models
主要内容：本文提出了一种广义长度外推方法，旨在提高预训练语言模型在处理长文本时的性能。作者通过对模型进行特定任务微调，实现了在多种任务上的长度泛化能力。实验结果表明，该方法在各种长度外推任务上均取得了较好的效果。
链接：https://arxiv.org/abs/2103.12436

4. 论文标题：Length Extrapolation with Sparse Attention and Continuous Paths
主要内容：本文通过结合稀疏注意力机制和连续路径技术，提出了一种新的长度外推方法。实验证明，该方法可以在不增加计算复杂度的情况下，有效提高模型在处理长文本时的性能。此外，论文还探讨了不同稀疏注意力策略对长度外推性能的影响。
链接：https://arxiv.org/abs/2106.05998

请注意，以上链接内容可能随着时间的推移而发生变化，请自行查阅相关论文以获取更多详细信息。
